In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [3]:
data = pd.read_csv("C:/Users/admin/Videos/london_merged.csv",parse_dates=[0], index_col=[0])

In [34]:
data.tail(3)

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
timestamp,,,,,,,,,
2017-01-01,6421,168.0,106.5,2122.5,339.0,150.0,0.0,24.0,72.0
2017-01-02,11823,89.5,22.5,1995.5,268.0,29.0,24.0,0.0,72.0
2017-01-03,19436,81.5,-15.0,1953.5,454.5,56.0,0.0,0.0,72.0


In [6]:
data = data.resample('D').sum()

In [7]:
df = data[['cnt']]

In [8]:
df.head(3)

,cnt
timestamp,
2015-01-04,9234
2015-01-05,20372
2015-01-06,20613


In [24]:
data_training = df[:584]['cnt']
data_test = df[584:]['cnt']

In [25]:
x_train = []
y_train = []

In [26]:
for i in range(4, len(data_training)-4):
    x_train.append(data_training[i-4:i])
    y_train.append(data_training[i:i+4])

In [27]:
x_train = np.array(x_train)

In [28]:
y_train = np.array(y_train)

In [30]:
x_train.shape, y_train.shape

((576, 4), (576, 4))

In [32]:
x_scaler = MinMaxScaler()
x_train = x_scaler.fit_transform(x_train)
y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(y_train)

In [52]:
x_train = x_train.reshape(576,4,1)

In [53]:
reg = Sequential()

reg.add(LSTM(units=200, return_sequences=True, input_shape=(4,1)))
reg.add(Dropout(0.2))

reg.add(LSTM(units=200, return_sequences=True, input_shape=(4,1)))
reg.add(Dropout(0.15))

reg.add(LSTM(units=176, return_sequences=True, input_shape=(4,1)))
reg.add(Dropout(0.10))

reg.add(Dense(1))


In [54]:
reg.compile(loss='mse', optimizer='adam')

In [55]:
reg.fit(x_train, y_train, epochs=200, batch_size=4)

Train on 576 samples
Epoch 1/200
576/576 [==============================] - 14s 24ms/sample - loss: 0.0249
Epoch 2/200
576/576 [==============================] - 5s 8ms/sample - loss: 0.0162 1s - l - ETA: 0s - loss:
Epoch 3/200
576/576 [==============================] - 5s 8ms/sample - loss: 0.0160TA: 2s  -
Epoch 4/200
576/576 [==============================] - 4s 8ms/sample - loss: 0.0162 0s - loss
Epoch 5/200
576/576 [==============================] - 5s 8ms/sample - loss: 0.0161
Epoch 6/200
576/576 [==============================] - 4s 8ms/sample - loss: 0.0157 0
Epoch 7/200
576/576 [==============================] - 4s 8ms/sample - loss: 0.0158
Epoch 8/200
576/576 [==============================] - 5s 8ms/sample - loss: 0.0159 0s - loss: 0.
Epoch 9/200
576/576 [==============================] - 4s 8ms/sample - loss: 0.0161
Epoch 10/200
576/576 [==============================] - 4s 8ms/sample - loss: 0.0157
Epoch 11/200
576/576 [==============================] - 5s 8ms/sample - loss

In [56]:
x_test = []
y_test = []

In [57]:
for i in range(4, len(data_test)-4):
    x_test.append(data_test[i-4:i])
    y_test.append(data_test[i:i+4])

In [58]:
x_test , y_test = np.array(x_test), np.array(y_test)

In [59]:
x_test.shape, y_test.shape

((139, 4), (139, 4))

In [60]:
x_test = x_scaler.fit_transform(x_test)

In [61]:
x_test = x_test.reshape(139,4,1)

In [62]:
y_pred = reg.predict(x_test)

In [63]:
y_pred.shape

(139, 4, 1)

In [64]:
y_pred = y_pred.reshape(139,4)

In [65]:
y_pred = y_scaler.transform(y_pred)

In [66]:
from sklearn.metrics import mean_squared_error

In [67]:
def evaluate_model(y_test,y_pred):
    scores=[]
    
    for i in range(y_test.shape[1]):
        mse = mean_squared_error(y_test[:,i],y_pred[:,i])
        rmse = np.sqrt(mse)
        scores.append(rmse)
        
    return scores

In [69]:
evaluate_model(y_test,y_pred)

[29589.04205184969, 29370.364098450842, 29176.494753547184, 29006.566376041636]

In [73]:
np.std(y_test[3])

7970.571670212871

# Linear Regression model has significantly less error has compared with ML (machine Learning) model. 